# Estudo sobre a predição do tempo de hopitalização de pacientes com COVID-19 em 2021

Este notebook tem por finalidade a reprodução do [estudo](https://iris.paho.org/handle/10665.2/56251) realizado por SOUZA, Fernanda S. H. de et al.

Dicionário de dados: https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/pdfs/Dicionario_de_Dados_SRAG_Hospitalizado_19.09.2022.pdf

Este estudo transversal utilizou dados de pacientes hospitalizados com COVID-19 com idade ≥18 anos, esquema vacinal completo e diagnóstico positivo de infecção por SARS-CoV-2, coletados do banco de dados SIVEP-Gripe (Sistema de Informação da Vigilância Epidemiológica da Gripe) de 18 de janeiro a 15 de setembro de 2021.

## Tabela de atributos úteis para o estudo

|  Variável  |   Tipo   |           Descrição          |
| :--------- | -------- | ---------------------------- |
| DT_SIN_PRI |   Date   | Data de 1º sintomas do caso. |
| SG_UF_NOT  | Varchar2 | Unidade Federativa onde está localizada a Unidade que realizou a notificação. |
| CS_SEXO    | Varchar2 |      Sexo do paciente.       |
| NU_IDADE_N | Varchar2 | Idade informada pelo paciente quando não se sabe a data de nascimento. |
| TP_IDADE   | Varchar2 | Tipo/Idade considerando o intervalo entre a data de nascimento e a data dos primeiros sintomas |
| CS_GESTANT | Varchar2 | Idade gestacional da paciente. |
| CS_RACA    | Varchar2 |         Cor ou raça.         |
| CS_ESCOL_N | Varchar2 | Nível de escolaridade do paciente. |
| NOSOCOMIAL | Varchar2 | Caso de SRAG com infecção adquirida após internação. |
|   FEBRE    | Varchar2 | Paciente apresentou febre? |
|   TOSSE    | Varchar2 | Paciente apresentou tosse? |
|  GARGANTA  | Varchar2 | Paciente apresentou dor de garganta? |
|  DISPNEIA  | Varchar2 | Paciente apresentou dispneia? |
| DESC_RESP  | Varchar2 | Paciente apresentou desconforto respiratório? |
| SATURACAO  | Varchar2 | Paciente apresentou saturação O2< 95%? |
|  DIARREIA  | Varchar2 | Paciente apresentou diarreia? |
|   VOMITO   | Varchar2 | Paciente apresentou vômito? |
|   DOR_ABD  | Varchar2 | Paciente apresentou dor abdominal? |
|   FADIGA   | Varchar2 | Paciente apresentou fadiga? |
| PERD_OLFT  | Varchar2 | Paciente apresentou perda do olfato? |
| PERD_PALA  | Varchar2 | Paciente apresentou perda do paladar? |
| OUTRO_SIN  | Varchar2 | Paciente apresentou outro(s) sintoma(s)? |
|  PUERPERA  | Varchar2 | Paciente é puérpera ou parturiente (mulher que pariu recentemente – até 45 dias do parto)? |
| CARDIOPATI | Varchar2 | Paciente possui Doença Cardiovascular Crônica? |
| HEMATOLOGI | Varchar2 | Paciente possui Doença Hematológica Crônica? |
| SIND_DOWN  | Varchar2 | Paciente possui Síndrome de Down? |
|  HEPATICA  | Varchar2 | Paciente possui Doença Hepática Crônica? |
|    ASMA    | Varchar2 | Paciente possui Asma? |
|  DIABETES  | Varchar2 | Paciente possui Diabetes mellitus? |
| NEUROLOGIC | Varchar2 | Paciente possui Doença Neurológica? |
| PNEUMOPATI | Varchar2 | Paciente possui outra pneumopatia crônica? |
| IMUNODEPRE | Varchar2 | Paciente possui Imunodeficiência ou Imunodepressão (diminuição da função do sistema imunológico)? |
|   RENAL    | Varchar2 | Paciente possui Doença Renal Crônica? |
| OBESIDADE  | Varchar2 | Paciente possui obesidade? |
| OUT_MORBI  | Varchar2 | Paciente possui outro(s) fator(es) de risco? |
| VACINA_COV | Varchar  | Informar se o paciente recebeu vacina COVID-19, após verificar a documentação / caderneta. |
| DOSE_1_COV | Varchar  | Informar a data em que o paciente recebeu a 1ª dose da vacina COVID-19. |
| DOSE_2_COV | Varchar  | Informar a data em que o paciente recebeu a 2ª dose da vacina COVID-19. |
|  HOSPITAL  | Varchar2 | O paciente foi internado? |
| DT_INTERNA |   Date   | Data em que o paciente foi hospitalizado. |
|    UTI     | Varchar2 | O paciente foi internado em UTI? |
| SUPORT_VEN | Varchar2 | O paciente fez uso de suporte ventilatório? |
| RAIOX_RES  | Varchar2 | Informar resultado de Raio X de Tórax. |
| CLASSI_FIN | Varchar2 | Diagnóstico final do caso. |
|  EVOLUCAO  | Varchar2 | Evolução do caso. |
| DT_EVOLUCA |   Date   | Data da alta ou óbito. |

## Importações e configurações

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

In [2]:
# A base de dados originalmente conta com dezenas de variáveis. Listá-las desta forma facilita tanto 
# a selecção quanto em futuras análistes separadas.
SYMPTOMS = [
    "FEBRE", "TOSSE", "GARGANTA", 
    "DISPNEIA", "DESC_RESP", "SATURACAO", 
    "DIARREIA", "VOMITO", "DOR_ABD", 
    "FADIGA", "PERD_OLFT", "PERD_PALA", "OUTRO_SIN"
]

COMORBIDITIES = [
    "PUERPERA", "CARDIOPATI", "HEMATOLOGI", 
    "SIND_DOWN", "HEPATICA", "ASMA", 
    "DIABETES", "NEUROLOGIC", "PNEUMOPATI", 
    "IMUNODEPRE", "RENAL", "OBESIDADE", "OUT_MORBI"
]

DATES = [
    "DT_SIN_PRI", "DT_INTERNA", "DOSE_1_COV",
    "DOSE_2_COV", "DT_EVOLUCA"
]

DEMOGRAPHICS = [
    "CS_SEXO", "NU_IDADE_N", "TP_IDADE", 
    "CS_ESCOL_N", "CS_RACA", "SG_UF_NOT"
]

HOSPITAL = [
    "CS_GESTANT", "NOSOCOMIAL", "VACINA_COV",
    "HOSPITAL", "UTI", "SUPORT_VEN",
    "RAIOX_RES", "CLASSI_FIN", "EVOLUCAO",
    "PCR_SARS2", "AN_SARS2", "LAB_PR_COV"
]

## Compreensão dos dados

In [3]:
df = pd.read_csv(
    "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/2021/INFLUD21-10-04-2023.csv",
    # "data/INFLUD21-10-04-2023.csv",
    sep=";",
    encoding="ISO-8859-1",
    usecols=SYMPTOMS + COMORBIDITIES + DATES + DEMOGRAPHICS + HOSPITAL,
    low_memory=False
)

In [4]:
df.head()

,DT_SIN_PRI,SG_UF_NOT,CS_SEXO,NU_IDADE_N,TP_IDADE,CS_GESTANT,CS_RACA,CS_ESCOL_N,NOSOCOMIAL,FEBRE,...,PCR_SARS2,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,AN_SARS2,VACINA_COV,DOSE_1_COV,DOSE_2_COV,LAB_PR_COV
0,05/01/2021,BA,F,74,3,6,9,9.0,2.0,1.0,...,NaN,2.0,1.0,2.0,2.0,NaN,1.0,18/03/2021,15/04/2021,86 - COVID-19-CORONAVAC-SINOVAC/BUTANTAN
1,04/01/2021,MG,F,73,3,5,4,9.0,2.0,2.0,...,NaN,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN
2,07/01/2021,SP,M,1,3,6,4,NaN,2.0,1.0,...,NaN,2.0,1.0,2.0,2.0,NaN,1.0,14/12/2022,NaN,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC
3,03/01/2021,SP,M,37,3,6,1,9.0,2.0,1.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,06/01/2021,SP,M,54,3,6,1,3.0,2.0,1.0,...,NaN,2.0,2.0,2.0,2.0,1.0,2.0,NaN,NaN,NaN


In [5]:
print(f"Tamanho do dataset: {df.shape}")

Tamanho do dataset: (1732198, 49)


## Tratamentos dos dados

 Conversão das datas para DateTime no formato __dd/mm/YYYY__
 
 O parâmetro ``errors="coerce"`` nos permite tratar como valor ausente as datas que não 
foram corretamente convertidas.

In [6]:
for att_date in DATES:
    df[att_date] = pd.to_datetime(df[att_date], format='%d/%m/%Y', errors='coerce')

O primeiro corte para análise é quanto ao tempo. Serão aceitos somente os pacientes admitidos entre 18 de janeiro a 15 de setembro de 2021

In [7]:
df = df.loc[ (df.DT_INTERNA > "18/01/2021") & (df.DT_INTERNA < "15/09/2021") ]

In [8]:
print(f"Tamanho do dataset: {df.shape}")

Tamanho do dataset: (1385038, 49)


A idade dos pacientes é informada pela combinação das variáveis ``NU_IDADE_N`` e ``TP_IDADE``.

No caso de pacientes abaixo de 1 ano de idade, o TP_IDADE poderá ser 1 ou 2, sendo atribuído o valor na variável NU_IDADE_N em dias (caso TP_IDADE = 1) ou meses (caso TP_IDADE = 2).

Entretanto, como o subgrupo a ser trabalhado é de pacientes maiores de 18 anos, só serão admitidos valores ``TP_IDADE = 3``.

In [9]:
df.TP_IDADE.value_counts()

TP_IDADE
3    1351845
2      28571
1       4622
Name: count, dtype: int64

In [10]:
legal_age_filter = (df.TP_IDADE == 3) & (df.NU_IDADE_N >= 18)
df = df.loc[ legal_age_filter ]

In [11]:
print(f"Tamanho do dataset: {df.shape}")

Tamanho do dataset: (1291139, 49)


As próximas filtragens foram de pacientes hospitalizados, diagnóstico positivo de infecção por SARS-CoV-2, informações sobre o sexo biológico do paciente não ausentes,

In [12]:
filtering  = (df.HOSPITAL == 1) & (df.CLASSI_FIN == 5) & (df.CS_SEXO.isin(["M", "F"]))
filtering &= ((df.PCR_SARS2 == 1) | (df.AN_SARS2 == 1))
df = df.loc[ filtering ]

In [13]:
print(f"Tamanho do dataset: {df.shape}")

Tamanho do dataset: (791828, 49)


### Exclusão de duplicatas
Pela incapacidade de identificar unicamente cada paciente dentro do dataset, será aplicado uma remoção simples de dados duplicados.

In [14]:
df.drop_duplicates(inplace=True)

In [15]:
print(f"Tamanho do dataset: {df.shape}")

Tamanho do dataset: (791822, 49)


### Verificação e limpeza de dados ausentes
Existir dados ausentes nas informações sobre sintomas ou comorbidade não é um problema por si só. Esses dados serão tratados como a falta de determinada característica. 

In [16]:
df[ DATES + DEMOGRAPHICS + HOSPITAL ].isna().sum()

DT_SIN_PRI         0
DT_INTERNA         0
DOSE_1_COV    644176
DOSE_2_COV    711670
DT_EVOLUCA     59311
CS_SEXO            0
NU_IDADE_N         0
TP_IDADE           0
CS_ESCOL_N    255410
CS_RACA            0
SG_UF_NOT          0
CS_GESTANT         0
NOSOCOMIAL    104810
VACINA_COV    188555
HOSPITAL           0
UTI            49588
SUPORT_VEN     59252
RAIOX_RES     311757
CLASSI_FIN         0
EVOLUCAO       30421
PCR_SARS2     181043
AN_SARS2      588085
LAB_PR_COV    644286
dtype: int64

### Exclusão de atributos redundantes ou desnecessários.

Nesta etapa do processo de limpeza, alguns atributos já serviram seu propósito, tais como ``TP_IDADE``, ``PCR_SARS2``, ``AN_SARS2``, etc.

Para evitar processamento de dados desnecessários, essas variáveis serão excluídas do dataframe.

In [17]:
df.drop(columns=[
    "PCR_SARS2", 
    "AN_SARS2",
    "TP_IDADE",
    "CLASSI_FIN",
], inplace=True)

In [18]:
print(f"Tamanho do dataset: {df.shape}")

Tamanho do dataset: (791822, 45)


### Transformação das variáveis
O dataset contém valores para presença (1), ausência (2) e não preenchimento (9) das variáveis de sintomas, comorbidades entre outras. A fim de facilitar o entendimento, esses dados serão transformados para o tipo ``bool`` e a ausência de preenchimento será tratada como __False__.

In [19]:
# Conversão de sintomas e comorbidades em bool
for variable in SYMPTOMS + COMORBIDITIES + ["NOSOCOMIAL"]:
    df[variable] = df[variable].map(lambda x: x == 1).astype(bool)

In [20]:
# Verificação de inconsistências nas datas de vacinação
filtering  = (df.DOSE_1_COV >= df.DOSE_2_COV) # 2ª vacinação antes da 1ª
filtering |= ((df.DOSE_1_COV.isna()) & (df.DOSE_2_COV.notna())) # 2ª vacinação sem a 1ª
df[ filtering ].shape

(3218, 45)

Existem 3218 registros de pessoas com inconsistências nas datas de vacinação, seja pela data da 2ª dose ser antes do que a da primeira ou a data da 1ª ser inexistente, mesmo havendo sido tomado 2 doses.

A ocorrência do primeiro caso (data da 2ª vacinação > 1ª vacinação) é, de fato, um problema de informação. É possível inferir que houve duas vacinações porém, a falta de confiança no processo de registro impede uma futura padronização do status de vacinação.

Desta forma, quaisquer dados com os problemas citados serão removido.

In [21]:
df = df[ ~filtering ]

In [22]:
print(f"Tamanho do dataset: {df.shape}")

Tamanho do dataset: (788604, 45)


- Transformações dos nomes das vacinas

In [23]:
df.LAB_PR_COV.nunique()

3279

As vacinas presentes nesse período no Brasil eram Coronavac, Oxford, Pfizer e Janssen. Porém, há 3279 diferentes tipos de entradas nesse campo, o que indica uma alta taxa de dados incorretos e uma falta de padronização.

Esta etapa é, sem dúvidas, a mais trabalhosa neste projeto. A abordagem a ser adotada será a separação por casamento exato de caracteres.

* Nomes oficiais e populares das vacinas

Nome oficial | Nome popular |
-------------|--------------|
Coronavac    | Sinovac, Butantan |
Oxford       | Fiocruz, Astrazeneca, Covishield, Biomanguinho, Oswaldo Cruz |
Pfizer       | Comirnaty, Biontech, |
Janssen      | Johnson & Johnson |


In [45]:
# Pfizer
df.loc[ df.LAB_PR_COV.str.contains('PF|ZER|FER|SER|COMIN|ATY|BIO|TECH|PZI|PZHIER', case=False, regex=True, na=False), "LAB_PR_COV"] =  "Pfizer"

In [46]:
# Janssen
df.loc[ df.LAB_PR_COV.str.contains('JAN|JOHNSON|SEN|JOH|SON', case=False, regex=True, na=False), "LAB_PR_COV"] = "Janssen"

In [53]:
# Coronavac
df.loc[ df.LAB_PR_COV.str.contains('CORO|VAC|CORANAC|SINO|SIVONAC|INSTITUTO|BUT|BUNT|NTAN|IB|I.B|BTT', case=False, regex=True, na=False), "LAB_PR_COV"] = "Coronavac"

In [64]:
# Oxford
df.loc[ df.LAB_PR_COV.str.contains('OXF|OSW|ORD|FIO|CRUZ|FC|AST|AZTRA|COVIS|ELD|SHILD|SHIEL|GUINHO|NECA|NICA|ZENCA|ADOX|VCD|VCD1', case=False, regex=True, na=False), "LAB_PR_COV"] = "Oxford"

In [61]:
# Nenhuma vacina informada
df.loc[ df.LAB_PR_COV.str.contains('NAO|SEM|TOMOU|DOSE|INVESTIGAR|SAUDE|/2021|/21|IMUNIZADO|SPUTINIK|DADOS|INFOR|COVAX|SARS CORV-2|LABORATORIO|-|/', case=False, regex=True, na=False), "LAB_PR_COV"] = ""

In [70]:
# Remoção das demais entradas que não condizem com nomes de vacinas
df["LAB_PR_COV"] = np.where(
    df.LAB_PR_COV == "Pfizer",
    "Pfizer",
    np.where(
        df.LAB_PR_COV == "Janssen",
        "Janssen",
        np.where(
            df.LAB_PR_COV == "Coronavac",
            "Coronavac",
            np.where(
                df.LAB_PR_COV == "Oxford",
                "Oxford",
                "Missing"
            )
        )
    )
)

In [71]:
df.LAB_PR_COV.value_counts()

LAB_PR_COV
Missing      644046
Coronavac     69504
Oxford        59760
Pfizer        13389
Janssen        1905
Name: count, dtype: int64

### Classificação do status de vacinação